In [4]:
import igl
from meshplot import plot
import numpy as np
import pygeodesic.geodesic as geodesic
import networkx as nx
import drawSvg as draw

EPSILON = 1e-5

# find out if new vertex is on the given line segment defined by the two vertices of the edge
def is_vertex_on_line_segment (vertex_A, vertex_B, new_vertex_C):
    result = False
    side_AB = vertex_A - vertex_B
    side_AC = vertex_A - new_vertex_C
    cross_ABAC = np.cross(side_AB, side_AC)
    # if cross product is zero then point is on line, check dot product to find if point lines on the "line segment"
    if np.linalg.norm(cross_ABAC) < EPSILON: #  check if all elements in array are zero
        dot_ABAB = np.dot(side_AB, side_AB)
        dot_ABAC = np.dot(side_AB, side_AC)
        # if dot product is greater than zero but less dot product of line segment with itself point is on the line segment
        if dot_ABAC > 0 and dot_ABAC < dot_ABAB:
            result = True
    return result

# goes through all edges of the mesh and returns the vertex ids of the edge that is intersected by the new vertex
def find_cut_edge_vertex_ids(vertices, faces, new_vertex):
    edges=igl.edges(faces)
    result = False
    for i, edge in enumerate(edges):
        result = is_vertex_on_line_segment(vertices[edge[0]], vertices[edge[1]], new_vertex)
        if result is True:
            cut_edge_vertices = [edge[0], edge[1]]
            break
    return cut_edge_vertices

# goes through the face list and finds the two faces that each have both cut vertex ids
def find_faces_shared_by_cut_edge (cut_edge_vertices, faces):
    first_face = None
    for i, face in enumerate(faces):
        if cut_edge_vertices[0] in face and cut_edge_vertices[1] in face:
            if first_face is None:
                first_face = i
            else:
                return first_face, i
    raise Exception ('did not find face {}'.format (cut_edge_vertices))

# Splits the two faces into two new faces and adds them to the list, deletes the old ones
def cut_face_in_two (faces, shared_face_ids, cut_edge_vertices, new_vertex_id):
    cut_face_1 = faces [shared_face_ids[0]]
    # Repeat for the other shared face
    cut_face_2 = faces [shared_face_ids[1]]
    
    # create copies of the orignal face
    face_1 = np.copy(cut_face_1)
    face_2 = np.copy(cut_face_1)
    face_3 = np.copy(cut_face_2)
    face_4 = np.copy(cut_face_2)
    
    # Replace the edge vertices with the new vertex in each copy
    face_1[face_1 == cut_edge_vertices[0]] = new_vertex_id
    face_2[face_2 == cut_edge_vertices[1]] = new_vertex_id
    face_3[face_3 == cut_edge_vertices[0]] = new_vertex_id
    face_4[face_4 == cut_edge_vertices[1]] = new_vertex_id
    
    # Delete the orignal face and append the new faces
    faces = np.delete(faces, [shared_face_ids[0],shared_face_ids[1]], axis = 0)
    faces = np.append(faces, [face_1], axis = 0)
    faces = np.append(faces, [face_2], axis = 0)
    faces = np.append(faces, [face_3], axis = 0)
    faces = np.append(faces, [face_4], axis = 0)

    return faces

#----------functions from the previous lecture "edge unfolding polyhedra"-----------

# adds edges based on the adjacency of faces in the mesh
def add_edges_from_mesh(graph, faces):
    face_adjacency_matrix, _ = igl.triangle_triangle_adjacency(faces)
    for face_id, _ in enumerate(faces):
        for ajd_face_id in face_adjacency_matrix[face_id]:
            graph.add_edge(face_id,ajd_face_id)
            

# adds nodes to graph G for every face in mesh
def add_nodes_from_mesh(graph, faces): [graph.add_node(face_id) for face_id, face in enumerate(faces)]

# create networkx graph from given mesh
def graph_from_mesh(faces):
    graph = nx.Graph()
    add_nodes_from_mesh(graph, faces)
    add_edges_from_mesh(graph, faces)
    return graph

In [5]:
raw_vertices, raw_faces = igl.read_triangle_mesh("cube.stl")
vertices, faces, _ = igl.remove_duplicates(raw_vertices, raw_faces, 0.00001)

# Define the source id and end id with respect to the vertex array
source_id = 0
end_id = len(vertices)
geodesic_paths = []

# Loop through all the vertices of the mesh as target ids
for i in range (1, end_id):
        # Define the target vertex id with respect to the vertex array
        target_id = i
        # add the vertex id of the target vertex to the current geodesic path
        geodesic_path = [target_id]
        # Initialise the PyGeodesicAlgorithmExact class instance
        # Reinitialise with update mesh after each path is created otherwise incorrect edges will be created
        geoalg = geodesic.PyGeodesicAlgorithmExact(vertices, faces)
        # Compute the geodesic distance and the path. 
        _, path = geoalg.geodesicDistance(source_id, target_id)  #  path is in order from target to source.
        # Faces are cut only if paths are longer than two vertices
        if len(path) > 2:
            # ignore the first and last vertex as they are already in the array
            for vertex_id in range (1, len(path)-1):
                # add path vertex to mesh vertex list
                vertices = np.append(vertices, [path[vertex_id]], axis=0)
                # obtain vertex id of newly added vertex
                new_vertex_id = len(vertices) - 1
                # find the vertex ids of the edge that the new vertex is located on
                cut_edge_vertices = find_cut_edge_vertex_ids(vertices, faces, path[vertex_id])
                # find the two faces that share the edge
                shared_face_ids = find_faces_shared_by_cut_edge (cut_edge_vertices, faces)
                # create two new faces/face and add them to the face list. Delete the two orignal faces
                faces = cut_face_in_two (faces, shared_face_ids, cut_edge_vertices, new_vertex_id)
                # add the vertex id of the newly created vertex to the current geodesic path
                geodesic_path.append(new_vertex_id)
        # add the vertex id of the source vertex to the current geodesic path
        geodesic_path.append(source_id)
        geodesic_paths.append(geodesic_path)

# initialize a networkx graph from the faces
graph = graph_from_mesh(faces)

for path in geodesic_paths:
    for i in range(len(path)-1):
        cut_edge_vertices = [path[i], path[i+1]]
        #use "find_faces_shared_by_cut_edge" to find the shared faces between them 
        #use graph.remove_edge(face_id1,face_id2) to delete the edges
        #check if graph is tree using “nx.is_tree(graph)” 

print(geodesic_paths)
        
p = plot(vertices, faces, return_plot=True)
p.add_points(vertices, shading={"point_color": "black","point_size": 0.4})
p.add_edges(vertices, faces, shading={"line_color": "red"});



[[1, 0], [2, 0], [3, 0], [4, 8, 9, 10, 0], [5, 11, 12, 0], [6, 13, 0], [7, 0]]


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…